In [1]:
from collections import defaultdict
import numpy as np
import pandas as pd
from pandas import DataFrame
import torch
from torch import cat, eq, int64, long, randperm, tensor, Tensor, zeros_like
from torch.utils.data import Dataset , DataLoader
import csv
import os


from dataset import *
from model import *
from utils import *

/home/ahora/anaconda3/lib/python3.9/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/home/ahora/anaconda3/lib/python3.9/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:93: UnderReviewWarning: The feature FeatureMapContrastiveTask is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  contrastive_task: Union[FeatureMapContrastiveT

In [2]:
torch.set_float32_matmul_precision('medium' )

In [3]:
dm = KGDataModule(batch_size=1024 * 8)

In [4]:

model_hidden_dim = 512
model_margin = 5
model = KGEmbedding(

    hidden_dim=model_hidden_dim, margin=model_margin,

    entity_filepath='hetionet/entity2id.tsv', relation_filepath='hetionet/relation2id.tsv',

    function_type="transE"
)


In [5]:

from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import LearningRateMonitor


logger = TensorBoardLogger("tb_logs", name="my_model")
lr_monitor = LearningRateMonitor(logging_interval='epoch')

callbacks = [
    ModelCheckpoint(
        save_top_k=1, mode="max", monitor="MRR"
    ),

    lr_monitor,
]


In [6]:

import lightning as L

from pytorch_lightning import Trainer, seed_everything

seed_everything(123)


trainer = L.Trainer(
    max_epochs=1000,
    callbacks=callbacks,
    accelerator="gpu",
    devices=1,
    precision="16-mixed",
    logger=logger,
    check_val_every_n_epoch=20,
)

trainer.fit(model, datamodule=dm)


Global seed set to 123
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


item of Datasets 1800949 224603 224645


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ahora/Desktop/Hetionet/model.py:171: UnderReviewWarning: The feature LinearWarmupCosineAnnealingLR is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  scheduler = LinearWarmupCosineAnnealingLR(

  | Name | Type | Params
------------------------------
------------------------------
23.1 M    Trainable params
0         Non-trainable params
23.1 M    Total params
92.533    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/home/ahora/anaconda3/lib/python3.9/site-packages/lightning/pytorch/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
# from lightning.pytorch.tuner import Tuner
# tuner = Tuner(trainer)
# tuner.scale_batch_size(model, datamodule=dm)

In [ ]:
# model.predict_tail("Gene::55025" , "PCiC")

In [7]:
torch.save(model.state_dict() , "./save_ahora.pt")